In [7]:
import cx_Oracle
import requests
from bs4 import BeautifulSoup

url2 = "https://quasarzone.com/bbs/qb_free"  # 크롤링 대상 URL
url_code = "02"  # 해당 URL 코드(URL_LIST.CODE_URL)

def db_insert(titles, counts, days, links): # DB 저장 함수(크롤링 추출 자료 INSET)
    cx_Oracle.init_oracle_client(lib_dir=r'C:\instantclient-basic-windows.x64-21.9.0.0.0dbru\instantclient_21_9')
    conn = cx_Oracle.connect(user='admin', password='Tongbrown@23', dsn='tongbrown_high')

    print('')
    print('DB connect 성공!!')
    print('')

    cs = conn.cursor()
    sql = 'INSERT INTO TONG_BROWN_ROW (CODE_URL,DATA_TITLE,DATA_COUNT,DATA_LINK,DATA_WRITE) VALUES (:1,:2,:3,:4,:5)'

    i = 0
    
    for title in titles:
        cs.execute(sql, (url_code, title.get_text(), counts[i].get_text().strip(), links[i].get('href'), days[i].get_text().strip()))
        i = i + 1

    print("[%d]건 INSERT 완료" % 30)
    cs.close()
    conn.commit()
    conn.close()

def url_parsing(): # 크롤링 함수
    # requests를 이용하여 URL의 HTML 문서를 가져옴
    response2 = requests.get(url2)

    # URL 접속이 되지 않으면 실행되지 않는다.
    if response2.status_code == 200: # 정상 상태코드(200)
        soup = BeautifulSoup(response2.text, "html.parser")

        # 해당 URL 추출 data를 구분한다.
        div = soup.select_one('div.dabate-type-list')

        titles = div.select('table > tbody > tr > td > p > a')
        counts = div.select('table > tbody > tr > td > span.count')
        days = div.select('table > tbody > tr > td > span.date')
        links = div.select("a", class_="subject-link")

        i = 0

        for title in titles:
            print("제목 : " + title.get_text().strip())
            print("조회수 : " + counts[i].get_text().strip())
            print("날짜 : " + days[i].get_text().strip())
            print("링크 " + links[i].get('href'))
            print('-------------------------------------------------------------------------')
            i=i+1
            
        db_insert(titles, days, counts, links)

if __name__ == '__main__':
    url_parsing()

제목 : KING 향해 나폴리가 움직인다! 김민재 이적설 맞힌 伊 기자, "이강인...
조회수 : 8
날짜 : 00:02
링크 /bbs/qb_free/views/8178246
-------------------------------------------------------------------------
제목 : 빅스마일데이 카드 할인 최대금액이 바뀌었네요.
조회수 : 89
날짜 : 05-14
링크 /bbs/qb_free/views/8178239
-------------------------------------------------------------------------
제목 : 해외의 결제용 통합 QR코드 들
조회수 : 54
날짜 : 05-14
링크 /bbs/qb_free/views/8178209
-------------------------------------------------------------------------
제목 : 혹시 닉 언급은 안되죠?
조회수 : 156
날짜 : 05-14
링크 /bbs/qb_free/views/8178202
-------------------------------------------------------------------------
제목 : 특가 게시판에 댓글작성이 막혀있네요
조회수 : 91
날짜 : 05-14
링크 /bbs/qb_free/views/8178197
-------------------------------------------------------------------------
제목 : 라이젠 9  7900 또 샀어요
조회수 : 166
날짜 : 05-14
링크 /bbs/qb_free/views/8178190
-------------------------------------------------------------------------
제목 : 모니터 어제 받아서  방금 설치했는데
조회수 : 132
날짜 : 05-14
링크 /bbs/qb_free/views/8178189
--------

ProgrammingError: Oracle Client library has already been initialized